In [ ]:
%matplotlib notebook
from pylab import *
import numpy as np
from craco.search_pipeline import location2pix, get_grid_lut_from_plan, grid_candidates,wait_for_starts,waitall,print_candidates
from craco.search_pipeline import NBINARY_POINT_FDMTIN 
import craco
import craco.search_pipeline
from craco.pyxrtutil import *
import craft.craco_plan

from craft import uvfits

from craft.craco_plan import PipelinePlan
from craft.craco import printstats
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
import time

In [ ]:
!ls -lrt /data/craco/ban115/builds/

In [ ]:
fits='/data/craco/ban115/test_data/frb_d0_t0_a1_sninf_lm00/frb_d0_t0_a1_sninf_lm00.fits'
xclbin='/data/craco/ban115/builds/binary_container_28db073f.xclbin'
#xclbin='/data/craco/ban115/builds/binary_container_01482863.xclbin'
xclbin='/data/craco/ban115/builds/binary_container_fe3fef0d.xclbin'

alloc_device_only=True
values = craco.search_pipeline.get_parser().parse_args([])
print(values)

In [ ]:
device = pyxrt.device(0)
xbin = pyxrt.xclbin(xclbin)
uuid = device.load_xclbin(xbin)

In [ ]:
f = uvfits.open(fits)
# TOOU UNDERSTAN DFOR NEXT WEEK: Why does specifying this list of DMs give different results  than the arange?
dms=[399,400,511] # need 511 so it's equivalent in allr espects to plan
#dms = np.arange(512, dtype=np.int32)

#dms = None
plan_small = PipelinePlan(f, '', dms=dms)
plan_large = PipelinePlan(f, '--ndm 512')


In [ ]:
plan = plan_small

In [ ]:
plan.nd

In [ ]:
plan.dmax

In [ ]:
plan_small.dmax

In [ ]:
plan.dms[:10]

In [ ]:
figure()
plot(plan.dms,'o')

In [ ]:
p = craco.search_pipeline.Pipeline(device, xbin, plan, alloc_device_only)

In [ ]:
from Visibility_injector.inject_in_fake_data import FakeVisibility
fvfile = '/data/craco/gup037/test_runs_of_craco_pipeline/injections/repitetive_dm_401_injection.yml'
fv = FakeVisibility(plan, fvfile)
fvblk = fv.get_fake_data_block()
nblk = 12

In [ ]:
cat $fvfile

In [ ]:
inblocks = [next(fvblk) for i in range(nblk)]

In [ ]:
inblocks = np.array(inblocks)

In [ ]:
inblocks.shape

In [ ]:
inblocksum = inblocks.sum(axis=(1,2,3))
inblocksum.reshape(-1,2)

In [ ]:
assert np.all(inblocks[::2, ...] == inblocks[0,...])
assert np.all(inblocks[1::2, ...] == inblocks[1,...])

In [ ]:
p.clear_buffers(values)
values.run_fdmt = True
values.run_image = True
values.threshold = 10
values.input_scale = 64
values.fft_scale1 = 2
cands = []
idm  = 400
pix = 0

bchist = []
mbhist = []

mb0 = p.all_mainbufs[6]

for iblk in range(nblk):
    p.copy_input(inblocks[iblk,...], values)
    p.run(iblk, values).wait()
    cands.append(p.get_candidates().copy())
    print(f'Got {len(cands[-1])} candidates')
    p.boxcar_history.copy_from_device()
    bchist.append(p.boxcar_history.nparr[idm,:,128,128].copy())
    #p.[83, 186, 11, 256, 8, 2]
    mb0.copy_from_device()
    mbhist.append(mb0.nparr[0,:,:,:,0,0].copy())
    
    
        

In [ ]:
bchist = np.array(bchist)
mbhist = np.array(mbhist)

In [ ]:
[len(c) for c in cands]

In [ ]:
maxcand = [max(c, key=lambda x: x['snr']) for c in cands if len(c) != 0]
maxcand

In [ ]:
maxcand[0].dtype

In [ ]:
all_cands = []
for c in cands:
    all_cands.extend(c)
    
snrs = [c['snr'] for c in all_cands]
figure()
hist(snrs)

In [ ]:
mbhist.shape

In [ ]:
figure()
imshow(mbhist.reshape(12*186,-1), aspect='auto', origin='lower')
xlabel('Ringbuffer slot x NT')
ylabel('Block x DM')

In [ ]:
import craco_pybind11
import craco_pybind11.ddgrid_reader
import craco_pybind11.grid
import craco_pybind11.fdmt_tunable

nchunk_time = plan.nchunk_time
nparallel_uvin, nparallel_uvout, h_nparallel_uvout, lut = get_grid_lut_from_plan(plan)
nuvrest = nparallel_uvin*2//8
NUVWIDE = 8
OUTPUT_NT = 2
OUTPUT_NUV = 2
ncu = 4
ndm = plan.nd
tblk = 0
do_load_lut = 1

In [ ]:
craco_pybind11.fdmt_tunable.NTBLK_OUTBUF

In [ ]:
craco_pybind11.krnl.NT

In [ ]:
tblks = (9,10,0)

outputs = np.zeros((len(tblks), ncu, ndm, nchunk_time, nuvrest, NUVWIDE//OUTPUT_NUV, OUTPUT_NT, OUTPUT_NUV, 2), dtype=np.int16)

In [ ]:
mainbuf = p.copy_mainbuf()

In [ ]:

for itblk, tblk in enumerate(tblks):
    craco_pybind11.ddgrid_reader.krnl_ddgrid_reader_4cu(mainbuf, 
                                     ndm, 
                                     tblk,
                                     nchunk_time, 
                                     nuvrest, 
                                     plan.ddreader_lut, 
                                     do_load_lut, 
                                     outputs[itblk, 0], 
                                     outputs[itblk, 1], 
                                     outputs[itblk, 2], 
                                     outputs[itblk, 3])

In [ ]:
plan.nd

In [ ]:
outputs.shape

In [ ]:
transposeidxs = [0, 3,1,6,2,4,5,7,8]
outputsrs = outputs.copy().transpose(transposeidxs)
outputsrs.shape

In [ ]:
s = np.array(outputsrs.shape)

outputsrs = outputsrs.reshape(s[0], s[1:4].prod(),s[4],s[5:8].prod(),s[8])
outputsrs.shape

In [ ]:
np.all(outputs == 0)

In [ ]:
np.all(outputsrs == 0)

In [ ]:
craco_pybind11.krnl.NT

In [ ]:
craco_pybind11.krnl.NUVWIDE

In [ ]:
np.all(outputsrs[0,...] == outputsrs[2,...])

In [ ]:
fig, ax = subplots(1,len(tblks))
for itblk, tblk in enumerate(tblks):
    ax[itblk].imshow(outputsrs[itblk,:,:,0,0].T, aspect='auto', origin='lower')
    ax[itblk].set_title(f'tblk={tblk}')

In [ ]:
outputsrs.shape

In [ ]:
fig, ax = subplots(1,len(tblks))
fig.set_size_inches((10,5))
for itblk, tblk in enumerate(tblks):
    ax[itblk].imshow(outputsrs[itblk,:,1,:,0], aspect='auto', origin='lower')
    ax[itblk].set_title(f'Tblk {tblk}')
    ax[itblk].set_xlabel('UV')
    ax[itblk].set_ylabel('time')
    
         

In [ ]:
outputsrs.shape

In [ ]:
outputsrs.shape[-2]/2

In [ ]:
figure()
imshow(outputsrs[0,:,1,:,0]-outputsrs[2,:,1,:,0], aspect='auto', origin='lower')

In [ ]:
mainbuf.shape

In [ ]:
figure()
imshow(mainbuf[0,:,:,:,0,0].reshape(186,-1), aspect='auto', origin='lower')
for b in range(11):
    axvline(b*256,c='r', alpha=0.2)

In [ ]:
mainbufrs = mainbuf[0:1,...].transpose([0,4,1,2,3,5]).reshape(-1,186,11*256,2)
mainbufrs.shape

In [ ]:
figure()
imshow(mainbufrs[0,:,:,0],aspect='auto',origin='lower')

In [ ]:
from craco_pybind11.ddgrid_reader import load_luts

luts_large = load_luts(plan_large.ddreader_lut[:-1])
luts_large

In [ ]:
luts_small = load_luts(plan_small.ddreader_lut[:-1])
luts_small

In [ ]:
from craco_pybind11.ddgrid_reader import calc_indexs
dmidx = 400
uvrest = 0
r = calc_indexs(dmidx, uvrest, luts_large)
print(f"IDM={r.idm} tback={r.tback} for idm={plan_large.dms[dmidx]}")

In [ ]:
dmidx = 2
r = calc_indexs(dmidx, uvrest,luts_small)
print(f"IDM={r.idm} tback={r.tback} for idm={plan_small.dms[dmidx]}")

In [ ]:

plan_small.dms[1:2]

In [ ]:
fig, ax = subplots(1,3)
for _, small_dm in enumerate(plan_small.dms[1:2]):
    small_dmidx = np.where(plan_small.dms == small_dm)[0][0]
    large_dmidx = np.where(plan_large.dms == small_dm)[0][0]
    print(small_dmidx, plan_small.dms[small_dmidx], large_dmidx, plan_large.dms[large_dmidx])
    assert plan_small.dms[small_dmidx] == plan_large.dms[large_dmidx]
    idx_small = [calc_indexs(small_dmidx, u, luts_small) for u in range(plan.nuvrest)]
    idx_large = [calc_indexs(large_dmidx, u, luts_large) for u in range(plan.nuvrest)]
    idx_all = np.array([(r1.idm, r2.idm, r1.tback, r2.tback) for (r1, r2) in zip(idx_small, idx_large)])
    ax[0].plot(idx_all[:,0])
    ax[0].plot(idx_all[:,1])
    ax[1].plot(idx_all[:,2])
    ax[1].plot(idx_all[:,3])
    ax[2].scatter(idx_all[:,0], idx_all[:,2])
    ax[0].set_title('IDM')
    ax[1].set_title('TBACK')
    ax[2].set_xlabel('IDM')
    ax[2].set_ylabel('TBACK')
    ax[0].set_xlabel('UVREST')
    ax[1].set_xlabel('UVREST')
    assert np.all(idx_all[:,0] == idx_all[:,1])
    assert np.all(idx_all[:,2] == idx_all[:,3])

In [ ]:
figure()
plot(plan.ddreader_lut, label='plan');
plot(plan_small.ddreader_lut-plan_large.ddreader_lut*0, label='plan_small');
legend()

In [ ]:
from craco_pybind11.ddgrid_reader import calc_burst_parameters
calc_burst_parameters?

In [ ]:
tblk = 0
tchunk = 0
idx_small = [calc_indexs(small_dmidx, u, luts_small) for u in range(plan.nuvrest)]
burst_params = [calc_burst_parameters(idx.idm, idx.tback, tblk, tchunk, u) for u, idx in enumerate(idx_small)]

In [ ]:
uvrest = 320
idx_small[uvrest].tback

In [ ]:
idx_small[uvrest].idm

In [ ]:
b = burst_params[uvrest]

In [ ]:
for f in dir(b):
    if not f.startswith('__'):
        print(f, getattr( b,f))
tmod = (b.idx1 - b.idx_bufstart)*2
tmod

In [ ]:
mainbuf.shape

In [ ]:
np.ravel_multi_index((320,45,0,0,0,0), mainbuf.shape)//16//2

In [ ]:
burst_idx1 = np.array([b.idx1 for b in burst_params])
burst_aligned = np.array([b.aligned for b in burst_params])
burst_idx_bufstart = np.array([b.idx_bufstart for b in burst_params])
burst_nread1 = np.array([b.nread1 for b in burst_params])
burst_nread2 = np.array([b.nread2 for b in burst_params])

In [ ]:
figure()
plot(burst_idx1-burst_idx_bufstart)
ylabel('index from first sample in block')
xlabel('urest')

# well that doesn't look right - it wraps to 1285 - shouldn't it wrap to twice that number?

In [ ]:
u = 320
tblk = 0
idx = idx_small[u]
burst_params = [calc_burst_parameters(idx.idm, idx.tback, tblk, tchunk, u) for tchunk in range(32)]
burst_idx1 = np.array([b.idx1 for b in burst_params])
burst_aligned = np.array([b.aligned for b in burst_params])
burst_idx_bufstart = np.array([b.idx_bufstart for b in burst_params])
burst_nread1 = np.array([b.nread1 for b in burst_params])
burst_nread2 = np.array([b.nread2 for b in burst_params])

In [ ]:
idx.tback

In [ ]:
figure()
plot(burst_idx1 - burst_idx_bufstart)

In [ ]:
assert np.all(burst_idx_bufstart == burst_idx_bufstart[0])

In [ ]:
plan_small.dms

In [ ]:
idm = [run.idm_cff for run  in plan.fdmt_plan.runs]
idm_small = [run.idm_cff for run in  plan_small.fdmt_plan.runs]
figure()
plot(idm, label='plan')
plot(idm_small, label='small')
legend()

In [ ]:
off = [run.offset_cff for run  in plan.fdmt_plan.runs]
off_small = [run.offset_cff for run in  plan_small.fdmt_plan.runs]
figure()
plot(off, label='plan')
plot(off_small, label='small')
legend()

In [ ]:
craco_pybind11.krnl.NUREST

In [ ]:
craco_pybind11.krnl.MAX_NDM